# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Предварительные задачи

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [ ]:
from collections import Counter
import multiprocessing as mp

In [ ]:
%%file count_letters.py
from collections import Counter
def count_letters(file):
    with open(file) as fp:
        text = fp.read().lower()
    return Counter(text)

In [ ]:
from count_letters import count_letters

In [ ]:
%%time
count_letters('./data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt')
count_letters('./data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt')

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

In [ ]:
%%time
if __name__ == '__main__': #чтобы выполнение кода не зациклилось
    files = ['./data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt', 
             './data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt']

    with mp.Pool(processes=len(files)) as pool: #заявка на количество процессов
        counters = pool.map(count_letters, files) #map - применение функции к каждому элементу последовательности, который мы передадим
    #отличие map тут в том, что он раскадывает на процессы
    #далее он ждет, пока все процессы завершатся и добавит результаты в список
    #на этом конец, если работаем в обычном .py файле, НО, если работаем с юпитером, то оно может зациклиться,пишем main
        

In [ ]:
type(counters)

In [ ]:
counters[0]

## Лабораторная работа 10

In [1]:
import requests
from collections import Counter
import multiprocessing as mp
import csv

In [20]:
url = 'https://downloader.disk.yandex.ru//disk/2db8f621ff7b385d0e906f61d777e1848db63e6e09b73de440c69e25f504eec1/61a6d9c7/0oFllvTGFfdpIBeA8vFr8dAEw5veVGL0VYhgZvCxYbaPLStzbCcHQLOwDC56-gudN_kSGSSjEcnl5gbTS2uS1w%3D%3D'
r = requests.get(url, allow_redirects=True)
open('recipes_full.csv', 'wb').write(r.content)

48

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [ ]:
with open('./data/recipes_full.csv', encoding='utf-8') as buff:
    quantity = sum(1 for el in buff)

file=open('./data/recipes_full.csv', encoding='utf-8')
element = csv.reader(file)
next(element)

numb = 1
for_file = ['id;tag;n_steps']

for i, j in enumerate(element):
    id_s = j[1]
    tag_s = j[5][2:-2].split(", ")
    n_steps = j[6]
    for tag in tag_s:
        for_file.append(f'{id_s};{tag};{n_steps}') 
    if i >= quantity // 8 * numb:
        file = open(f'id_tag_nsteps_{numb}.csv', 'w', encoding='utf-8')
        file.write('\n'.join(for_file))
        file.close()
        print(f'Файл id_tag_nsteps_{numb}.csv был создан')
        numb += 1
        for_file = ['id;tag;n_steps']

In [21]:
import csv

n_f_ws = 8
f_ws = [f'id_tag_nsteps_{i}.csv' for i in range(n_f_ws)]
with open('recipes_full.csv') as f:  
    reader = csv.DictReader(f)
    for n, row in enumerate(reader):
        f_w = f_ws[n%n_f_ws]
        r = [row['id'], row['tags'], row['n_steps']]
        with open(f_w, 'a+', newline='') as w:
            csv.writer(w).writerow(r)

2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

In [11]:
from collections import defaultdict
import ast

def get_mean_steps_by_tag(fname):
  tags_and_steps = defaultdict(list)
  with open(fname) as f:  
    reader = csv.reader(f)
    for row in reader:
      tags = ast.literal_eval(row[1])
      steps = int(row[2])
      for tag in tags:
        tags_and_steps[tag].append(steps)
  del tags_and_steps['']
  tags_and_steps = dict(zip(
    list(tags_and_steps.keys()), 
    list(map(lambda x: sum(tags_and_steps[x])/len(tags_and_steps[x]), tags_and_steps))
  ))
  return tags_and_steps

get_mean_steps_by_tag('id_tag_nsteps_0.csv')

{'mexican': 4.5,
 'healthy-2': 6.0,
 'orange-roughy': 4.0,
 'chicken-thighs-legs': 5.0,
 'freezer': 6.666666666666667,
 'whitefish': 4.0,
 'pork-sausage': 4.0,
 'filipino': 4.0,
 'for-large-groups': 7.0,
 'pasta-salad': 4.0,
 'rosh-hashana': 4.75,
 'cambodian': 5.5,
 'pasta': 6.75,
 'fruit': 4.0,
 'cabbage': 6.0,
 'grains': 6.0,
 'equipment': 9.375,
 'lime': 5.0,
 'low-sodium': 5.333333333333333,
 'bass': 5.0,
 'meatballs': 3.0,
 'veal': 3.0,
 'prepared-potatoes': 3.5,
 'oaxacan': 3.5,
 'collard-greens': 3.5,
 'pheasant': 6.0,
 'fudge': 6.0,
 'micro-melanesia': 6.0,
 'cajun': 5.0,
 'breakfast-eggs': 6.0,
 'honduran': 5.0,
 'halloween-cocktails': 5.0,
 'beef': 5.8,
 'irish': 3.5,
 'turkey': 8.333333333333334,
 'Throw the ultimate fiesta with this sopaipillas recipe from Food.com.': 3.3333333333333335,
 'gluten-free': 3.6666666666666665,
 'pickeral': 3.3333333333333335,
 'south-american': 4.0,
 'black-bean-soup': 2.0,
 'pasta-rice-and-grains': 5.75,
 'food-processor-blender': 2.0,
 'brit

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


In [ ]:
%%time

from collections import defaultdict
import ast

def get_steps_and_tag(fname):
  print('Файл в процессе:', fname)
  tags_and_steps = defaultdict(list)
  with open(fname) as f:  
    reader = csv.reader(f)
    for row in reader:
      tags = ast.literal_eval(row[1])
      steps = int(row[2])
      for tag in tags:
        tags_and_steps[tag].append(steps)
  del tags_and_steps['']
  print('Файл  обработан:', fname)
  return tags_and_steps

def disp_all_files(n=8):
  tags_and_steps = defaultdict(list)
  for i in range(n):
    cur = get_steps_and_tag(f'id_tag_nsteps_{i}.csv')
    for key, value in cur.items():
      tags_and_steps[key] += value
  tags_and_steps = dict(zip(
    list(tags_and_steps.keys()), 
    list(map(lambda x: sum(tags_and_steps[x])/len(tags_and_steps[x]), tags_and_steps))
  ))
  return tags_and_steps

print(list(disp_all_files().items())[:10])

Файл в процессе: id_tag_nsteps_0.csv
Файл  обработан: id_tag_nsteps_0.csv
Файл в процессе: id_tag_nsteps_1.csv
Файл  обработан: id_tag_nsteps_1.csv
Файл в процессе: id_tag_nsteps_2.csv
Файл  обработан: id_tag_nsteps_2.csv
Файл в процессе: id_tag_nsteps_3.csv
Файл  обработан: id_tag_nsteps_3.csv
Файл в процессе: id_tag_nsteps_4.csv
Файл  обработан: id_tag_nsteps_4.csv
Файл в процессе: id_tag_nsteps_5.csv
Файл  обработан: id_tag_nsteps_5.csv
Файл в процессе: id_tag_nsteps_6.csv
Файл  обработан: id_tag_nsteps_6.csv
Файл в процессе: id_tag_nsteps_7.csv
Файл  обработан: id_tag_nsteps_7.csv
[('mexican', 5.302344316442439), ('healthy-2', 6.384162244806188), ('orange-roughy', 3.513425052701653), ('chicken-thighs-legs', 4.145581465931509), ('freezer', 4.033042234819468), ('whitefish', 3.514734127201888), ('pork-sausage', 4.256068444090729), ('filipino', 3.575355140695586), ('for-large-groups', 7.292883853009813), ('pasta-salad', 3.5048206710374084)]
CPU times: user 50.2 s, sys: 1.26 s, total: 5

4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

In [ ]:
%%time

from collections import defaultdict
import ast
import multiprocessing

def get_steps_and_tag(fname):
  print('Файл в процессе:', fname)
  tags_and_steps = defaultdict(list)
  with open(fname) as f:  
    reader = csv.reader(f)
    for row in reader:
      tags = ast.literal_eval(row[1])
      steps = int(row[2])
      for tag in tags:
        tags_and_steps[tag].append(steps)
  del tags_and_steps['']
  print('Файл  обработан:', fname)
  return tags_and_steps

def disp_all_files(n=8, proc_n=2):
  tags_and_steps = defaultdict(list)
  fnames = [f'id_tag_nsteps_{i}.csv' for i in range(n)]
  all = multiprocessing.Pool(processes=proc_n).map(get_steps_and_tag, fnames)
  for cur in all:
    for key, value in cur.items():
      tags_and_steps[key] += value
  tags_and_steps = dict(zip(
    list(tags_and_steps.keys()), 
    list(map(lambda x: sum(tags_and_steps[x])/len(tags_and_steps[x]), tags_and_steps))
  ))
  return tags_and_steps

print(list(disp_all_files().items())[:10])

Файл в процессе: id_tag_nsteps_0.csv
Файл в процессе: id_tag_nsteps_1.csv
Файл  обработан: id_tag_nsteps_1.csv
Файл  обработан: id_tag_nsteps_0.csv
Файл в процессе: id_tag_nsteps_2.csv
Файл в процессе: id_tag_nsteps_3.csv
Файл  обработан: id_tag_nsteps_2.csv
Файл в процессе: id_tag_nsteps_4.csv
Файл  обработан: id_tag_nsteps_3.csv
Файл в процессе: id_tag_nsteps_5.csv
Файл  обработан: id_tag_nsteps_4.csv
Файл в процессе: id_tag_nsteps_6.csv
Файл  обработан: id_tag_nsteps_5.csv
Файл в процессе: id_tag_nsteps_7.csv
Файл  обработан: id_tag_nsteps_6.csv
Файл  обработан: id_tag_nsteps_7.csv
[('mexican', 5.302344316442439), ('healthy-2', 6.384162244806188), ('orange-roughy', 3.513425052701653), ('chicken-thighs-legs', 4.145581465931509), ('freezer', 4.033042234819468), ('whitefish', 3.514734127201888), ('pork-sausage', 4.256068444090729), ('filipino', 3.575355140695586), ('for-large-groups', 7.292883853009813), ('pasta-salad', 3.5048206710374084)]
CPU times: user 1.26 s, sys: 992 ms, total: 2

5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 

In [ ]:
%%time

from collections import defaultdict
import ast
import multiprocessing

def get_steps_and_tag(q_in, q_out):
  while not q_in.empty():
    print('Размер  очереди:', q_in.qsize())
    fname = q_in.get()
    print('Файл в процессе:', fname)
    tags_and_steps = defaultdict(list)
    with open(fname) as f:  
      reader = csv.reader(f)
      n = 0
      for row in reader:
        tags = ast.literal_eval(row[1])
        steps = int(row[2])
        for tag in tags:
          tags_and_steps[tag].append(steps)
    del tags_and_steps['']
    print('Файл  обработан:', fname)
    q_out.put(tags_and_steps)
  return True

def disp_all_files(n=8, proc_n=2):
  tags_and_steps = defaultdict(list)
  q_out = multiprocessing.Queue()
  q_in = multiprocessing.Queue()
  fnames = [f'id_tag_nsteps_{i}.csv' for i in range(n)]
  _ = [q_in.put(fname) for fname in fnames]
  ps = [multiprocessing.Process(target=get_steps_and_tag, args=(q_in, q_out,)) for _ in range(proc_n)]
  _ = [p.start() for p in ps]
  all = [q_out.get() for _ in fnames]
  for cur in all:
    for key, value in cur.items():
      tags_and_steps[key] += value
  tags_and_steps = dict(zip(
    list(tags_and_steps.keys()), 
    list(map(lambda x: sum(tags_and_steps[x])/len(tags_and_steps[x]), tags_and_steps))
  ))
  return tags_and_steps

print(list(disp_all_files().items())[:10])

Размер  очереди: 8
Файл в процессе: id_tag_nsteps_0.csv
Размер  очереди: 7
Файл в процессе: id_tag_nsteps_1.csv
Файл  обработан: id_tag_nsteps_1.csv
Размер  очереди: 6
Файл в процессе: id_tag_nsteps_2.csv
Файл  обработан: id_tag_nsteps_0.csv
Размер  очереди: 5
Файл в процессе: id_tag_nsteps_3.csv
Файл  обработан: id_tag_nsteps_2.csv
Размер  очереди: 4
Файл в процессе: id_tag_nsteps_4.csv
Файл  обработан: id_tag_nsteps_3.csv
Размер  очереди: 3
Файл в процессе: id_tag_nsteps_5.csv


In [ ]:
import time

def he(q_in, q_out):
  while not q_in.empty():
    print('Размер  очереди:', q_in.qsize())
    time.sleep(1)
    fname = q_in.get()
    q_out.put(fname*2)
  return True


q_out = multiprocessing.Queue()
q_in = multiprocessing.Queue()
fnames = [1,2,3]
_ = [q_in.put(fname) for fname in fnames]
ps = [multiprocessing.Process(target=he, args=(q_in, q_out,)) for _ in range(2)]
for p in ps:
  p.daemon = True
_ = [p.start() for p in ps]
# _ = [p.join() for p in ps]
all = [q_out.get() for _ in fnames]
print(all)

Размер  очереди: 3
Размер  очереди: 3
Размер  очереди: 2
Размер  очереди: 1
[2, 4, 6]
